In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Database connection parameters
username = 'postgres'
password = '1111'
host = 'localhost'
port = 5433
database = 'online-school'

# Creating connection string
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Creating an engine object
engine = create_engine(connection_string)

# Reading data from a table into a DataFrame
table_name = 'courses'
query = f'SELECT * FROM {table_name}'

# Executing the query and reading data into a DataFrame
courses = pd.read_sql(query, engine)

# Displaying the first 5 rows of the DataFrame
courses

,id,course_name,category,start_date,price
0,1,Introduction to Programming,Programming,2023-07-01 09:00:00,500.0
1,2,Advanced Data Science,Data Science,2023-08-15 10:00:00,750.0
2,3,Digital Marketing Basics,Marketing,2023-09-10 11:00:00,300.0
3,4,Project Management Essentials,Management,2023-07-20 09:00:00,400.0
4,5,Effective Communication Skills,Soft Skills,2023-10-05 09:30:00,250.0
...,...,...,...,...,...
191,192,Integrated Marketing Communications,Marketing,2021-03-25 09:30:00,400.0
192,193,Ruby Programming,Programming,2022-05-05 09:00:00,650.0
193,194,Usability Testing,Quality Assurance,2016-11-10 09:00:00,450.0
194,195,Negotiation Strategies,Soft Skills,2021-02-10 10:00:00,300.0


In [3]:
import random
from datetime import datetime
from faker import Faker

fake = Faker()  # Initialize the Faker instance to generate fake data
Faker.seed(0)  # Seed the random generator for reproducibility

# Initialize lists and sets
students = []  # List to store student records
used_phone_numbers = set()  # Set to ensure unique phone numbers

# Generate student data
for i in range(5136):  # Loop to generate data for 5136 students
    id = i + 1  # Assign a unique ID starting from 1
    first_name = fake.first_name()  # Generate a fake first name
    last_name = fake.last_name()  # Generate a fake last name
    
    # Assign gender with 2% probability for 'Non-binary'
    gender = random.choices(
        ['Male', 'Female', 'Non-binary'], 
        weights=[49, 49, 2], 
        k=1
    )[0]
    
    # Generate a date of birth between 1970 and 2000
    date_of_birth = fake.date_between(
        start_date=pd.to_datetime("1970-01-01"), 
        end_date=pd.to_datetime("2000-01-01")
    ).strftime('%Y-%m-%d')
    
    # Generate a unique phone number
    phone = fake.numerify(text='###-###-####')
    while phone in used_phone_numbers:  # Ensure phone number is unique
        phone = fake.numerify(text='###-###-####')
    used_phone_numbers.add(phone)  # Add the unique phone number to the set
    
    email = fake.email()  # Generate a fake email
    country = fake.country()  # Generate a fake country
    
    # Create a dictionary with all the student data
    student = {
        'id': id,
        'first_name': first_name,
        'last_name': last_name,
        'gender': gender,
        'date_of_birth': date_of_birth,
        'phone': phone,
        'email': email,
        'country': country
    }
    students.append(student)  # Add the student record to the list

# Create DataFrame from the list of students
students_df = pd.DataFrame(students)

# Print the DataFrame to verify the generated data
students_df

,id,first_name,last_name,gender,date_of_birth,phone,email,country
0,1,Megan,Chang,Male,1984-04-23,048-764-7593,juancampos@example.net,South Georgia and the South Sandwich Islands
1,2,John,Browning,Female,1993-12-29,924-115-7815,ostewart@example.org,Micronesia
2,3,Thomas,Grimes,Non-binary,1978-11-12,080-160-9753,jane13@example.net,Cyprus
3,4,John,Ponce,Non-binary,1997-04-30,871-158-7148,jasmine85@example.net,Costa Rica
4,5,Tamara,Morrison,Male,1989-12-29,471-965-9342,lorijohnson@example.net,Eritrea
...,...,...,...,...,...,...,...,...
5131,5132,Zachary,Mcneil,Female,1983-10-12,775-504-7116,michael16@example.com,Australia
5132,5133,Jeffrey,Allen,Male,1992-09-17,259-909-2177,wilsoneric@example.org,San Marino
5133,5134,Wesley,Benjamin,Male,1975-10-29,587-875-2067,kwilliams@example.com,Luxembourg
5134,5135,Matthew,Johnson,Male,1977-06-17,494-934-1942,jesse92@example.org,Uruguay


In [5]:
from itertools import product

fake = Faker()  # Initialize the Faker instance for generating random dates

# Prepare the course_student table and generate unique combinations of courses and students
course_student = courses[['id', 'start_date']].rename(columns={'id': 'course_id'})  # Rename 'id' to 'course_id'
unique_combinations = list(product(courses['id'], students_df['id']))  # Generate all possible course-student combinations

random.shuffle(unique_combinations)  # Shuffle the combinations to randomize the assignment

# Create a dictionary to track the number of unique courses assigned to each student
student_course_counts = {student_id: 0 for student_id in students_df['id']}

# Create a dictionary to track the required number of courses for each student (random between 1 and 32)
required_courses_per_student = {student_id: random.randint(1, 32) for student_id in students_df['id']}

# Initialize a list to store the final course-student assignments
final_data = []

# Assign courses to students based on their required number of courses
for course_id, student_id in unique_combinations:
    if student_course_counts[student_id] < required_courses_per_student[student_id]:
        student_course_counts[student_id] += 1  # Increment the count of courses assigned to the student
        final_data.append({'course_id': course_id, 'student_id': student_id})  # Add the course-student pair to the final data

# Convert final_data to a DataFrame and drop duplicates to ensure unique course-student pairs
final_df = pd.DataFrame(final_data).drop_duplicates(subset=['course_id', 'student_id'])

# Merge the course_student table with the final_df to get the full list of courses assigned to students
course_student = course_student.merge(final_df, on='course_id')

# Convert all dates to tz-naive (remove timezone information)
course_student['start_date'] = course_student['start_date'].dt.tz_localize(None)

# Function to generate a random purchase date based on the course start date
def generate_date_of_purchase(row):
    if row['start_date'] > pd.Timestamp('2024-07-01'):
        return fake.date_between_dates(date_start=pd.Timestamp('2024-01-01'), date_end=pd.Timestamp('2024-07-01'))
    else:
        return fake.date_between_dates(date_start=row['start_date'], date_end=pd.Timestamp('2024-07-01'))

# Apply the date generation function to each row to create the 'date_of_purchase' column
course_student['date_of_purchase'] = course_student.apply(generate_date_of_purchase, axis=1)

course_student  # Display the final course-student DataFrame

,course_id,start_date,student_id,date_of_purchase
0,1,2023-07-01 09:00:00,4622,2024-06-29
1,1,2023-07-01 09:00:00,3698,2023-09-13
2,1,2023-07-01 09:00:00,141,2024-03-22
3,1,2023-07-01 09:00:00,1643,2024-04-08
4,1,2023-07-01 09:00:00,1246,2024-06-27
...,...,...,...,...
83840,196,2017-08-15 09:00:00,2535,2018-02-13
83841,196,2017-08-15 09:00:00,3394,2022-11-01
83842,196,2017-08-15 09:00:00,2041,2021-06-24
83843,196,2017-08-15 09:00:00,2964,2017-12-16


In [7]:
course_student['date_of_purchase'] = pd.to_datetime(course_student['date_of_purchase'])  # Convert 'date_of_purchase' to datetime format

# Count the number of unique course_id for each combination of student_id and date_of_purchase
orders = (course_student
          .groupby(['student_id', 'date_of_purchase'])  # Group by student_id and date_of_purchase
          .agg(course_count=('course_id', 'count'))  # Count the number of courses purchased
          .reset_index()  # Reset the index for proper DataFrame formatting
          .sort_values(['student_id', 'date_of_purchase']))  # Sort by student_id and date_of_purchase

# Add a unique identifier column for each row
orders['id'] = np.arange(1, len(orders) + 1)  # Create a new 'id' column with sequential numbers

# Define probabilities for generating random discounts
probabilities = np.array([0.6, 0.2, 0.1, 0.05, 0.025, 0.01, 0.008, 0.007])  # Set probabilities for discount choices
probabilities /= probabilities.sum()  # Normalize the probabilities to ensure they sum to 1

# Define possible discount values and generate random discounts using the defined probabilities
discount_values = np.array(range(0, 71, 10))  # Define discount values (0 to 70 in steps of 10)
orders['discount'] = np.random.choice(discount_values, size=len(orders), p=probabilities)  # Assign random discounts based on probabilities

# Rearrange the columns in the desired order
orders = orders[['id', 'student_id', 'date_of_purchase', 'course_count', 'discount']].reset_index(drop=True)  # Reorder and reset index

orders  # Display the final DataFrame


,id,student_id,date_of_purchase,course_count,discount
0,1,1,2019-01-23,1,20
1,2,1,2019-11-03,1,20
2,3,1,2019-11-21,1,0
3,4,1,2020-04-27,1,0
4,5,1,2020-08-23,1,10
...,...,...,...,...,...
83197,83198,5136,2022-05-10,1,10
83198,83199,5136,2022-11-24,1,0
83199,83200,5136,2022-12-16,1,0
83200,83201,5136,2023-07-01,1,0


In [10]:
# Merge the course_student and orders DataFrames on 'student_id' and 'date_of_purchase' columns
course_order = pd.merge(
    course_student[['course_id', 'student_id', 'date_of_purchase']],  # Select relevant columns from course_student
    orders[['student_id', 'id', 'date_of_purchase']],  # Select relevant columns from orders
    on=['student_id', 'date_of_purchase']  # Perform merge on both 'student_id' and 'date_of_purchase'
)

# Rename 'id' to 'order_id' and remove duplicate rows
course_order = course_order.rename(columns={'id': 'order_id'}).drop_duplicates()  # Rename 'id' to 'order_id' and drop duplicates

# Select only the required columns and sort by 'order_id' and 'course_id'
course_order = course_order[['order_id', 'course_id', 'date_of_purchase']].sort_values(['order_id', 'course_id'])  # Reorder columns and sort

course_order  # Display the final DataFrame

,order_id,course_id,date_of_purchase
50066,1,117,2019-01-23
80590,2,189,2019-11-03
55379,3,130,2019-11-21
48670,4,114,2020-04-27
39788,5,93,2020-08-23
...,...,...,...
67192,83198,158,2022-05-10
64153,83199,151,2022-11-24
51556,83200,121,2022-12-16
59999,83201,141,2023-07-01


In [11]:
fake = Faker()

# Copy course_order to create a new DataFrame 'enrollments'
enrollments = course_order.copy()

# Function to generate enrollment data
def generate_dates(row):
    # Generate enrollment_date within the range from date_of_purchase to 2024-07-01
    enrollment_date = fake.date_between_dates(date_start=row['date_of_purchase'], date_end=pd.Timestamp('2024-07-01')) 
    
    # Generate last_visit_date between enrollment_date and 2024-07-01
    last_visit_date = fake.date_between_dates(date_start=enrollment_date, date_end=pd.Timestamp('2024-07-01'))
    
    # Calculate the number of days between enrollment and last visit
    days_elapsed = (last_visit_date - enrollment_date).days
    
    # Determine progress based on the number of days elapsed
    if days_elapsed < 30:
        progress = np.random.randint(0, 41)  # Less than 30 days, progress cannot exceed 40%
    else:
        if np.random.rand() < 0.1:
            progress = np.random.randint(0, 21)  # 10% chance of low progress even after 4-5 months
        else:
            progress = np.random.randint(40, 101)  # Otherwise, progress can be between 40% and 100%
    
    # Return a dictionary with the generated data
    return {
        'course_id': row['course_id'],
        'order_id': row['order_id'],
        'date_of_purchase': row['date_of_purchase'],
        'enrollment_date': enrollment_date,
        'last_visit_date': last_visit_date,
        'progress': progress
    }

# Generate new enrollment records by applying the generate_dates function to each row
new_records = [generate_dates(row) for _, row in course_order.iterrows()]
enrollments = pd.DataFrame(new_records)

# Merge the enrollments DataFrame with orders to get the student_id based on order_id
enrollments = pd.merge(enrollments, orders[['id', 'student_id']].rename(columns={'id': 'order_id'}), 
                       on='order_id', how='left')

# Randomly sample 87% of the records
enrollments = enrollments.sample(frac=0.87, random_state=1)

# Reset the index of the DataFrame
enrollments.reset_index(drop=True, inplace=True)

# Add a new 'id' column for each record
enrollments['id'] = np.arange(1, len(enrollments) + 1)

# Select only the necessary columns for the final DataFrame
enrollments = enrollments[['id', 'course_id', 'student_id', 'enrollment_date', 'last_visit_date', 'progress']]

enrollments  # Display the final DataFrame


,id,course_id,student_id,enrollment_date,last_visit_date,progress
0,1,82,2976,2023-02-21,2023-04-22,100
1,2,19,3633,2024-06-03,2024-06-04,5
2,3,193,657,2023-07-26,2023-08-24,23
3,4,13,2550,2024-06-27,2024-06-29,4
4,5,21,646,2024-01-21,2024-05-29,73
...,...,...,...,...,...,...
72940,72941,101,2894,2022-08-20,2023-02-17,61
72941,72942,21,2296,2024-05-10,2024-06-22,47
72942,72943,63,689,2024-01-07,2024-01-23,25
72943,72944,137,2762,2024-06-05,2024-06-15,12


In [12]:
# SQL query to create the 'enrollments' table if it doesn't exist
create_table_query = text('''
CREATE TABLE IF NOT EXISTS enrollments (
    id SERIAL PRIMARY KEY,                         
    course_id INTEGER NOT NULL,                     
    student_id INTEGER NOT NULL,                    
    enrollment_date TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,  
    last_visit_date TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP, 
    progress SMALLINT NOT NULL DEFAULT 0            
);
''')

# Execute the table creation query
with engine.connect() as conn:
    conn.execute(create_table_query)  # Create the table if it doesn't already exist

# Save the DataFrame 'enrollments' to the SQL table
# If the table exists, it will append the data. If not, it will create the table
enrollments.to_sql('enrollments', con=engine, index=False, if_exists='append')

print(f"DataFrame saved to table 'enrollments' in the database.")

DataFrame saved to table 'enrollments' in the database.


In [13]:
# Ensure 'course_order' only has the necessary columns
course_order = course_order[['course_id', 'order_id']]

# SQL query to create the 'course_orders' table if it doesn't exist
create_table_query = text('''
CREATE TABLE IF NOT EXISTS course_orders (
    id SERIAL PRIMARY KEY,           
    course_id INTEGER NOT NULL,   
    order_id INTEGER NOT NULL        
);
''')

# Execute the table creation query
with engine.connect() as conn:
    conn.execute(create_table_query)  # Create the table if it doesn't already exist

# Save the DataFrame 'course_order' to the SQL table
# If the table exists, it will append the data. If not, it will create the table
course_order.to_sql('course_orders', con=engine, index=False, if_exists='append')

print(f"DataFrame saved to table 'course_orders' in the database.")

DataFrame saved to table 'course_orders' in the database.


In [14]:
# Define the SQL query for creating the 'orders' table
create_table_query = text('''
CREATE TABLE IF NOT EXISTS orders (
    id SERIAL PRIMARY KEY,
    student_id INTEGER NOT NULL,
    date_of_purchase TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    course_count SMALLINT NOT NULL DEFAULT 1,
    discount SMALLINT NOT NULL DEFAULT 0
);
''')

# Execute the table creation query
with engine.connect() as conn:
    conn.execute(create_table_query)

# Save the 'orders' DataFrame to the 'orders' table in the database
orders.to_sql('orders', con=engine, index=False, if_exists='append')

print(f"DataFrame saved to table 'orders' in the database.")

DataFrame saved to table 'orders' in the database.


In [15]:
# SQL query to create a table with an auto-incrementing primary key
create_table_query = text('''
CREATE TABLE IF NOT EXISTS students (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    gender VARCHAR(20) CHECK (gender IN ('Male', 'Female', 'Non-binary')) NOT NULL,
    date_of_birth DATE NOT NULL,
    phone VARCHAR(15) NOT NULL,
    email VARCHAR(50) NOT NULL,
    country VARCHAR(60) NOT NULL
);
''')

# Execute the query to create the table
with engine.connect() as conn:
    conn.execute(create_table_query)

# Insert data into the existing table without passing the 'id' column (it will be auto-incremented)
students_df.to_sql('students', con=engine, index=False, if_exists='append')

print(f"DataFrame saved to table 'students' in the database.")

DataFrame saved to table 'students' in the database.


In [16]:
# Select relevant columns from enrollmments DataFrame to create reviews DataFrame
reviews = enrollments[['progress', 'course_id', 'student_id']].copy()

# Filter reviews where progress is greater than or equal to 80
reviews = reviews.query('progress >= 80') \
                 .reset_index(drop=True) \
                 .drop(columns=['progress'])  # Drop the progress column

# Generate random ratings from 1 to 5 for each review
reviews['rating'] = np.random.randint(1, 6, size=len(reviews))

# Reset index starting from 1 for better readability
reviews.index += 1

# Display the first 5 rows of the reviews DataFrame
reviews

,course_id,student_id,rating
1,82,2976,1
2,61,1668,3
3,191,946,4
4,162,4625,5
5,175,951,4
...,...,...,...
14516,52,3703,2
14517,111,834,5
14518,73,4108,5
14519,128,1152,1


In [17]:
# SQL query to create a table with an auto-incrementing primary key
create_table_query = text('''
CREATE TABLE IF NOT EXISTS reviews (
    id SERIAL PRIMARY KEY,
    student_id INTEGER NOT NULL,
    course_id INTEGER NOT NULL,
    rating SMALLINT NOT NULL DEFAULT 0
);
''')

# Execute the query to create the table
with engine.connect() as conn:
    conn.execute(create_table_query)

# Save DataFrame to SQL table (creates automatically if it does not exist)
reviews.to_sql('reviews', con=engine, index=False, if_exists='append')

print(f"DataFrame saved to table 'reviews' in the database.")

DataFrame saved to table 'reviews' in the database.


In [18]:
# Reading data from a table into a DataFrame
table_name = 'professors'
query = f'SELECT * FROM {table_name}'

# Executing the query and reading data into a DataFrame
professors = pd.read_sql(query, engine)

# Displaying the first 5 rows of the DataFrame
professors

,id,first_name,last_name,gender,date_of_birth,phone,email,education_level,start_date,end_date,is_active
0,1,Corbet,Kleinert,Male,1986-04-15,352-939-6063,ckleinert0@salon.com,DD,2018-07-14 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
1,2,Connie,Braney,Female,1982-09-10,702-928-5558,cbraney1@tuttocitta.it,MD,2019-02-19 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
2,3,Doll,Peddersen,Female,1988-03-15,179-476-1910,dpeddersen2@themeforest.net,BD,2020-11-29 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
3,4,Merle,Greenshiels,Female,1975-02-20,627-167-3684,mgreenshiels3@merriam-webster.com,MD,2017-05-09 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
4,5,Torey,Adie,Male,1980-08-20,351-323-5064,tadie4@comsenz.com,PhD,2015-09-30 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
...,...,...,...,...,...,...,...,...,...,...,...
194,195,Neila,Lackington,Female,1985-02-11,512-596-2725,nlackington27@noaa.gov,BD,2023-01-19 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
195,196,Margalit,Attoe,Female,1981-04-19,589-657-1333,mattoe28@wikia.com,MD,2023-02-14 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
196,197,Laina,MacCahee,Female,1986-06-13,791-456-5036,lmaccahee29@fastcompany.com,BD,2023-03-09 20:00:00+00:00,2999-01-01 00:00:00+04:00,True
197,198,Burtie,Dulwitch,Male,1982-07-17,478-350-9122,bdulwitch2a@harvard.edu,MD,2023-04-19 20:00:00+00:00,2999-01-01 00:00:00+04:00,True


In [21]:
# Initialize Faker for generating random data
fake = Faker()

# Convert date columns to datetime without time zones
courses['start_date'] = pd.to_datetime(courses['start_date']).dt.tz_localize(None)
professors['start_date'] = pd.to_datetime(professors['start_date']).dt.tz_localize(None)

# Create an empty list to store course assignments
course_assignments = []

# Iterate over each course
for _, course in courses.iterrows():
    # Select professors who started working at least 2 months before the course start date
    eligible_professors = professors[professors['start_date'] <= course['start_date'] - pd.DateOffset(months=2)]
    
    if not eligible_professors.empty:
        # Randomly choose one professor from the eligible list
        assigned_professor = eligible_professors.sample(n=1).iloc[0]
        
        # Generate an assignment date between the professor's start date and the day before the course start date
        assignment_date = fake.date_between_dates(
            date_start=assigned_professor['start_date'], 
            date_end=course['start_date'] - pd.DateOffset(days=1)
        )
        
        course_assignments.append({
            'course_id': course['id'],
            'professor_id': assigned_professor['id'],
            'assignment_date': assignment_date
        })
    else:
        print(f"No eligible professors for course {course['id']}")

# Convert the list of assignments to a DataFrame
course_assignments_df = pd.DataFrame(course_assignments)
# Adjust index to start from 1
course_assignments_df.index += 1
course_assignments_df

,course_id,professor_id,assignment_date
1,1,2,2021-06-02
2,2,115,2020-12-13
3,3,189,2022-09-07
4,4,25,2023-03-13
5,5,72,2022-05-27
...,...,...,...
192,192,120,2020-10-29
193,193,155,2020-10-07
194,194,24,2016-09-14
195,195,163,2020-11-01


In [23]:
# SQL query to create the 'course_assignments' table if it does not exist
create_table_query = text('''
CREATE TABLE IF NOT EXISTS course_assignments (
    id SERIAL PRIMARY KEY,
    course_id INTEGER NOT NULL,
    professor_id INTEGER NOT NULL,
    assignment_date TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);
''')

# Execute the SQL query to create the table
with engine.connect() as conn:
    conn.execute(create_table_query)

# Save the DataFrame to the SQL table, creating it automatically if it does not exist
course_assignments_df.to_sql('course_assignments', con=engine, index=False, if_exists='append')

print(f"DataFrame saved to table 'course_assignments' in the database.")

DataFrame saved to table 'course_assignments' in the database.


In [24]:
# Add a foreign key constraint to the 'course_assignments' table to reference 'courses' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE course_assignments
        ADD CONSTRAINT FK_course_assignments_courses
        FOREIGN KEY (course_id)
        REFERENCES courses (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

# Add a foreign key constraint to the 'course_assignments' table to reference 'professors' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE course_assignments
        ADD CONSTRAINT FK_course_assignments_professors
        FOREIGN KEY (professor_id)
        REFERENCES professors (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

In [25]:
# Add a foreign key constraint to the 'course_orders' table to reference the 'courses' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE course_orders
        ADD CONSTRAINT FK_course_orders_courses
        FOREIGN KEY (course_id)
        REFERENCES courses (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

# Add a foreign key constraint to the 'course_orders' table to reference the 'orders' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE course_orders
        ADD CONSTRAINT FK_course_orders_orders
        FOREIGN KEY (order_id)
        REFERENCES orders (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

In [26]:
# Add a foreign key constraint to the 'enrollments' table to reference the 'courses' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE enrollments
        ADD CONSTRAINT FK_enrollments_courses
        FOREIGN KEY (course_id)
        REFERENCES courses (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

# Add a foreign key constraint to the 'enrollments' table to reference the 'students' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE enrollments
        ADD CONSTRAINT FK_enrollments_students
        FOREIGN KEY (student_id)
        REFERENCES students (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

In [27]:
# Add a foreign key constraint to the 'orders' table to reference the 'students' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE orders
        ADD CONSTRAINT FK_orders_students
        FOREIGN KEY (student_id)
        REFERENCES students (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

In [28]:
# Add a foreign key constraint to the 'reviews' table to reference the 'courses' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE reviews
        ADD CONSTRAINT FK_reviews_courses
        FOREIGN KEY (course_id)
        REFERENCES courses (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )

# Add a foreign key constraint to the 'reviews' table to reference the 'students' table
with engine.connect() as conn:
    conn.execute(
        text('''
        ALTER TABLE reviews
        ADD CONSTRAINT FK_reviews_students
        FOREIGN KEY (student_id)
        REFERENCES students (id)
        ON UPDATE CASCADE
        ON DELETE CASCADE;
        ''')
    )